# DSPy Component Overview

In this notebook, we will explore the high level components of DSPy and build an intuition for how each of them work together.

## Notebook Setup

In [1]:
# Importing the necessary Python libraries
import os
import json
import yaml

import dspy

/home/vscode/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Loading my personal API keys from file (not pushed to GitHub due to .gitignore file)
if os.path.exists('../keys/api_keys.yaml'):
    with open('../keys/api_keys.yaml') as f:
        api_keys = yaml.safe_load(f)

## Language Model (LM)

In [3]:
# Setting the two different language models (LMs) we will work with
try:
    lm_4o_mini = dspy.LM('openai/gpt-4o-mini', api_key = os.environ['OPENAI_API_KEY'])
    lm_4o = dspy.LM('openai/gpt-4o', api_key = os.environ['OPENAI_API_KEY'])
except:
    lm_4o_mini = dspy.LM('openai/gpt-4o-mini', api_key = api_keys['OPENAI_API_KEY'])
    lm_4o = dspy.LM('openai/gpt-4o', api_key = api_keys['OPENAI_API_KEY'])

# Setting DSPy to use GPT-4o-mini as the default LM
dspy.configure(lm = lm_4o_mini)

## Signatures
In DSPy, a **signature** is more or less a blueprint to define what you expect to feed as an **input** versus what you expect to get back as an **output**. Given that our goal may be to optimize our prompt template, our signature does NOT need to contain a robust prompt template that you may be used to forming through trial-and-error experiments. We might provide details to the signature where things may be ambiguous, but by and large, we do not need a fully fledged prompt template to get started.

While we don't provide a full prompt template, we may need to supply the signature with a few details to help the model understand what we are specifically expecting. This may include...

- Setting the **data type** of the inputs / outputs
- Providing a **brief description** of what the input / output is
- Providing an expected **input / output structure** of what the inputs / outputs will consist of

DSPy offers two ways to create signatures: a **simpler string-based inline signature** and a **more robust class-based signature**. The simpler version is intended to offer a simple means to get going quickly; however, I personally am concerned with the lack of "rigidity" that the inline approach offers. Specifically, if you're using an IDE like VS Code, it won't be able to catch syntax errors in the inline version of the signature. Additionally, if you have a complex set of inputs and outputs, that can quickly get messy in the inline approach.

That said, I will demonstrate both approaches in this notebook, but in the other tutorials, you will see that I will be using the class-based approach.

(Note: In order to make this sample code work, we're going to have to make use of a simple **DSPy module** called `dspy.Predict()`. At this point in the tutorial, we have not yet covered modules, so please skip on down to that section if you want a better intuition on the code here.)

### Inline Signature
First, let's demonstrate how the inline signature works. We're going to keep it simple by creating a DSPy signature that expects any sentence but returns a Boolean value that represents the sentiment of the sentance. If the sentiment is positive, we will return True; otherwise, we will return False.

In [5]:
# Creating sample sentences representing positive and negative sentiment
positive_sentence = "I am very happy with the results of this project."
negative_sentence = "I am disappointed with the outcome of this task."

# Instantiating a simple DSPy module for sentiment classification
dspy_sentiment_classification = dspy.Predict('sentence -> sentiment: bool')

# Invoking the DSPy model with each respective sentence.
print(f'Positive sentence: {dspy_sentiment_classification(sentence = positive_sentence)}')
print(f'Negative sentence: {dspy_sentiment_classification(sentence = negative_sentence)}')

Positive sentence: Prediction(
    sentiment=True
)
Negative sentence: Prediction(
    sentiment=False
)


### Class-based Signature